In [14]:
import torchvision
import torch.nn as nn
import numpy as np
import torchvision.transforms as transforms

In [25]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
patch_path = '/content/drive/MyDrive/patch_outputs'

Mounted at /content/drive


In [36]:
import matplotlib.image as img
img.imread('/content/drive/MyDrive/patch_outputs/h&e_benign_case24_match1_patch0.png')

array([[[0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ],
        ...,
        [0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ]],

       [[0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ],
        [0.03529412, 0.02352941, 0.03137255],
        ...,
        [0.25490198, 0.10196079, 0.15294118],
        [0.20392157, 0.07843138, 0.12156863],
        [0.        , 0.        , 0.        ]],

       [[0.        , 0.        , 0.        ],
        [0.44313726, 0.3254902 , 0.4       ],
        [0.56078434, 0.40784314, 0.5058824 ],
        ...,
        [0.37254903, 0.15686275, 0.22745098],
        [0.29411766, 0.11372549, 0.17254902],
        [0.        , 0.        , 0.        ]],

       ...,

       [[0.        , 0.        , 0.        ],
        [0.05882353, 0.05882353, 0.05882353],
        [0.9137255 , 0

In [ ]:
num_classes = 2
model = torchvision.models.alexnet(pretrained = True)
# replace the last classifier
model.classifier[6] = nn.Linear(4096, num_classes)

In [ ]:
model.classifier

Sequential(
  (0): Dropout(p=0.5, inplace=False)
  (1): Linear(in_features=9216, out_features=4096, bias=True)
  (2): ReLU(inplace=True)
  (3): Dropout(p=0.5, inplace=False)
  (4): Linear(in_features=4096, out_features=4096, bias=True)
  (5): ReLU(inplace=True)
  (6): Linear(in_features=4096, out_features=2, bias=True)
)

In [ ]:
# Import necessary libraries
import torch
from PIL import Image
import torchvision.transforms as transforms

# Read a PIL image
image = Image.open('iceland.jpg')

# Define a transform to convert PIL
# image to a Torch tensor
transform = transforms.Compose([
    transforms.PILToTensor()
])

# transform = transforms.PILToTensor()
# Convert the PIL image to Torch tensor
img_tensor = transform(image)

# print the converted Torch tensor
print(img_tensor)
